In [1]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 
from scipy import interpolate


In [23]:
base_path = 'E:\\indoor-location-navigation\\'

In [24]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv(base_path + 'sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [4]:
# get only the wifi bssid that occur over 1000 times(this number can be experimented with)
# these will be the only ones used when constructing features
bssid = dict()

In [21]:


for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train\\'+building+'\\*')))
    print(building)
    wifi = list()
    for folder in folders:
        print(folder)
        floor = floor_map[folder.split('\\')[-1]]
        print(floor)
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            with open(file, encoding="utf8") as f:
                txt = f.readlines()
                for e, line in enumerate(txt):
                    tmp = line.strip().split()
                    if tmp[1] == "TYPE_WIFI":
                        wifi.append(tmp)
    df = pd.DataFrame(wifi)
    #top_bssid = df[3].value_counts().iloc[:500].index.tolist()
    value_counts = df[3].value_counts()
    top_bssid = value_counts[value_counts > 0].index.tolist()
    print(len(top_bssid))
    bssid[building] = top_bssid
    del df
    del wifi
    gc.collect()

5a0546857ecc773753327266
E:\indoor-location-navigation\train\5a0546857ecc773753327266\B1
-1
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F1
0
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F2
1
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F3
2
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F4
3
3397
5c3c44b80379370013e0fd2b
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\B1
-1
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F1
0
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F2
1
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F3
2
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F4
3
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F5
4
3063
5d27075f03f801723c2e360f
E:\indoor-location-navigation\train\5d27075f03f801723c2e360f\B1
-1
E:\indoor-location-navigation\train\5d27075f03f801723c2e360f\F1
0
E:\indoor-location-navigation\train\5d27075f03f801723c

In [22]:
with open("bssid_1000.json", "w") as f:
    json.dump(bssid, f)

In [5]:
with open("bssid_1000.json") as f:
    bssid = json.load(f)

In [5]:
# generate all the training data 
building_dfs = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train/', building +'/*')))
    dfs = list()
    index = sorted(bssid[building])
    print(building)
    for folder in folders:
        floor = floor_map[folder.split('\\')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            #print(file)
            wifi = list()
            waypoint = list()
            magnetic = list()
            accelerometer = list()
            with open(file, encoding="utf8") as f:
                txt = f.readlines()
            for line in txt:
                line = line.strip().split()
                if line[1] == "TYPE_WAYPOINT":
                    waypoint.append(line)
                if line[1] == "TYPE_WIFI":
                    wifi.append(line)
                if line[1] == "TYPE_MAGNETIC_FIELD":
                    magnetic.append(line)
                if line[1] == "TYPE_ACCELEROMETER":
                    accelerometer.append(line)
                

            df = pd.DataFrame(np.array(wifi))  
            df_magnet = pd.DataFrame(np.array(magnetic))
            df_magnet[0] = pd.to_numeric(df_magnet[0])
            df_magnet[2] = pd.to_numeric(df_magnet[2])
            df_magnet[3] = pd.to_numeric(df_magnet[3])
            df_magnet[4] = pd.to_numeric(df_magnet[4])
            
            df_accelerometer = pd.DataFrame(np.array(accelerometer))
            df_accelerometer[0] = pd.to_numeric(df_accelerometer[0])
            df_accelerometer[2] = pd.to_numeric(df_accelerometer[2])
            df_accelerometer[3] = pd.to_numeric(df_accelerometer[3])
            df_accelerometer[4] = pd.to_numeric(df_accelerometer[4])
            
            begining_timestamp = 0

            # generate a feature, and label for each wifi block
            for gid, g in df.groupby(0):
                dists = list()
                gid_int = int(gid)
                for e, k in enumerate(waypoint):
                    dist = abs(gid_int - int(k[0]))
                    dists.append(dist)
                nearest_wp_index = np.argmin(dists)
                
                #Identifying second waypoint timestamp ID so that wifi timestamp is between two waypoint timestamps
                if int(waypoint[nearest_wp_index][0]) > gid_int:
                    second_wp_index = nearest_wp_index -1
                else:
                    second_wp_index = nearest_wp_index +1
                
                if second_wp_index < 0:
                    second_wp_index = 0
                if second_wp_index >= len(dists):
                    second_wp_index = nearest_wp_index
                
                #print(int(waypoint[nearest_wp_index][0]), int(waypoint[second_wp_index][0]), gid_int)
                    
                    
                fx_interpolate = interpolate.interp1d([
                                                        int(waypoint[nearest_wp_index][0]),
                                                       int(waypoint[second_wp_index][0])
                                                      ],
                                                      [
                                                        float(waypoint[nearest_wp_index][2]),
                                                       float(waypoint[second_wp_index][2])
                                                      ])
                
                fy_interpolate = interpolate.interp1d([
                                                        int(waypoint[nearest_wp_index][0]),
                                                       int(waypoint[second_wp_index][0])
                                                      ],
                                                      [
                                                        float(waypoint[nearest_wp_index][3]),
                                                       float(waypoint[second_wp_index][3])
                                                      ])
                
                x_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< gid_int), 2].mean()
                y_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< gid_int), 3].mean()
                z_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< gid_int), 4].mean()
                
                x_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< gid_int), 2].mean()
                y_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< gid_int), 3].mean()
                z_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< gid_int), 4].mean()
                
                begining_timestamp = gid_int
                
                g = g.drop_duplicates(subset=3)
                tmp = g.iloc[:,3:5]
                feat = tmp.set_index(3).reindex(index).T
                
                
                feat['x_accel']=x_accel
                feat['y_accel']=y_accel
                feat['z_accel']=z_accel
                
                feat['x_magnet']=x_magnet
                feat['y_magnet']=y_magnet
                feat['z_magnet']=z_magnet
                
                try:
                    feat["x"] = fx_interpolate(gid_int)
                except:
                    feat["x"] = float(waypoint[nearest_wp_index][2])
                try:
                    feat["y"] = fy_interpolate(gid_int)
                except:
                    feat["y"] = float(waypoint[nearest_wp_index][3])
                feat["f"] = floor
                feat["path"] = file.split('/')[-1].split('.')[0] # useful for crossvalidation
                dfs.append(feat)
                
    building_df = pd.concat(dfs)
    building_dfs[building] = df
    building_df.to_csv(building+"_1000_train.csv")

5a0546857ecc773753327266


C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\inte

5c3c44b80379370013e0fd2b


C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\inte

5d27075f03f801723c2e360f


...

C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\inte

5dbc1d84c1eb61796cf7c010


C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\inte

5dc8cea7659e181adb076a3f


C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\interpolate\inte

In [8]:
ssubm_building_g = ssubm_df.groupby(0) #Grouped by building
feature_dict = dict()

for gid0, g0 in ssubm_building_g:
    index = sorted(bssid[g0.iloc[0,0]])
    feats = list()
    print(gid0, 'gid0')
#     print(g0, 'g0')
    #print()
    for gid,g in g0.groupby(1): #Grouping by path
        # get all wifi time locations, 
        #print(gid, 'gid')
        #print(g, 'g')
#       print()
        with open(os.path.join(base_path, 'test\\' + g.iloc[0,1] + '.txt'), encoding="utf8") as f:
            txt = f.readlines()

        wifi = list()
        magnetic = list()
        accelerometer = list()

        for line in txt:
            line = line.strip().split()
            if line[1] == "TYPE_WIFI":
                wifi.append(line)
            if line[1] == "TYPE_MAGNETIC_FIELD":
                magnetic.append(line)
            if line[1] == "TYPE_ACCELEROMETER":
                accelerometer.append(line)

        wifi_df = pd.DataFrame(wifi) #All wifi points for certain path
        df_magnet = pd.DataFrame(np.array(magnetic))
        df_magnet[0] = pd.to_numeric(df_magnet[0])
        df_magnet[2] = pd.to_numeric(df_magnet[2])
        df_magnet[3] = pd.to_numeric(df_magnet[3])
        df_magnet[4] = pd.to_numeric(df_magnet[4])
        
        
        df_accelerometer = pd.DataFrame(np.array(accelerometer))
        df_accelerometer[0] = pd.to_numeric(df_accelerometer[0])
        df_accelerometer[2] = pd.to_numeric(df_accelerometer[2])
        df_accelerometer[3] = pd.to_numeric(df_accelerometer[3])
        df_accelerometer[4] = pd.to_numeric(df_accelerometer[4])
        
        begining_timestamp = 0
        
        wifi_points = pd.DataFrame(wifi_df.groupby(0).count().index.tolist()) #Timestamps for wifi for certain path
        magnetic_points = pd.DataFrame(df_magnet.groupby(0).count().index.tolist()) #Timestamps for magnetometer for certain path
        
        #print(wifi_points)
        begining_timestamp=0
        for timepoint in g.iloc[:,2].tolist():
            timepoint_int = int(timepoint)
            deltas = (wifi_points.astype(int) - timepoint_int).abs()
            min_delta_idx = deltas.values.argmin()
            wifi_block_timestamp = wifi_points.iloc[min_delta_idx].values[0]
            
            
            
            x_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< timepoint_int), 2].mean()
            y_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< timepoint_int), 3].mean()
            z_magnet = df_magnet.loc[(df_magnet[0]>=begining_timestamp) & (df_magnet[0]< timepoint_int), 4].mean()
            
            
            x_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< timepoint_int), 2].mean()
            y_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< timepoint_int), 3].mean()
            z_accel = df_accelerometer.loc[(df_accelerometer[0]>=begining_timestamp) & (df_accelerometer[0]< timepoint_int), 4].mean()
                
            
            if x_magnet !=x_magnet:
                x_magnet = df_magnet.iloc[0, 2]
                y_magnet = df_magnet.iloc[0, 3]
                z_magnet = df_magnet.iloc[0, 4]
            if x_accel !=x_accel:
                x_accel = df_accelerometer.iloc[0, 2]
                y_accel = df_accelerometer.iloc[0, 3]
                z_accel = df_accelerometer.iloc[0, 4]
                #print(x_magnet, ' assigned')
            begining_timestamp = timepoint_int
            wifi_block = wifi_df[wifi_df[0] == wifi_block_timestamp].drop_duplicates(subset=3)
            feat = wifi_block.set_index(3)[4].reindex(index)
            
            feat['x_accel']=x_accel
            feat['y_accel']=y_accel
            feat['z_accel']=z_accel
            
            feat['x_magnet']=x_magnet
            feat['y_magnet']=y_magnet
            feat['z_magnet']=z_magnet
            
            feat['site_path_timestamp'] = g.iloc[0,0] + "_" + g.iloc[0,1] + "_" + timepoint
            feats.append(feat)
    feature_df = pd.concat(feats, axis=1).T
    feature_df.to_csv(gid0+"_1000_test.csv")
    feature_dict[gid0] = feature_df

5a0546857ecc773753327266 gid0
5c3c44b80379370013e0fd2b gid0
5d27075f03f801723c2e360f gid0
5d27096c03f801723c31e5e0 gid0
5d27097f03f801723c320d97 gid0
5d27099f03f801723c32511d gid0
5d2709a003f801723c3251bf gid0
5d2709b303f801723c327472 gid0
5d2709bb03f801723c32852c gid0
5d2709c303f801723c3299ee gid0
5d2709d403f801723c32bd39 gid0
5d2709e003f801723c32d896 gid0
5da138274db8ce0c98bbd3d2 gid0
5da1382d4db8ce0c98bbe92e gid0
5da138314db8ce0c98bbf3a0 gid0
5da138364db8ce0c98bc00f1 gid0
5da1383b4db8ce0c98bc11ab gid0
5da138754db8ce0c98bca82f gid0
5da138764db8ce0c98bcaa46 gid0
5da1389e4db8ce0c98bd0547 gid0
5da138b74db8ce0c98bd4774 gid0
5da958dd46f8266d0737457b gid0
5dbc1d84c1eb61796cf7c010 gid0
5dc8cea7659e181adb076a3f gid0


In [31]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import sklearn

In [32]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [33]:
feature_path = 'C:\\Users\\user\\Documents\\LightHouseLabs\\W09\\'

In [34]:
train_files = sorted(glob.glob(os.path.join(feature_path, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_path, '*_test.csv')))

In [35]:
le = sklearn.preprocessing.LabelEncoder()

In [ ]:
predictions = list()

for e, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[e], index_col=0)

    print(file)
    data = data.sample(frac=1) #Shuffling needed?
    
    data.fillna(-100, inplace=True)
    test_data.fillna(-100, inplace=True)
    
    
    x_trainX = data.iloc[:int(len(data)*1), :-4]
    y_trainy = data.iloc[:int(len(data)*1),-3]
    y_trainx = data.iloc[:int(len(data)*1),-4]
    y_trainf = data.iloc[:int(len(data)*1),-2] #- data['f'].min() #To avoid negative numbers and start from 0
#     x_testX = data.iloc[int(len(data)*0.9):, :-4]
#     y_testy = data.iloc[int(len(data)*0.9):,-3]
#     y_testx = data.iloc[int(len(data)*0.9):,-4]
#     y_testf = data.iloc[int(len(data)*0.9):,-2] - y_testf.min()
    
    y_trainf = le.fit_transform(y_trainf)
    
    #Keras first input layer
    print(x_trainX.shape[0], x_trainX.shape[1])
    
    normal_layer = preprocessing.Normalization()
    normal_layer.adapt(x_trainX.to_numpy())
    
    inputs = keras.Input(shape=(x_trainX.shape[1],), name='Input')
        
    x = normal_layer(inputs)
    # x = layers.Embedding(input_dim=x_trainX.shape[1], output_dim = 128)(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Reshape((1, -1))(x)
    x = layers.LayerNormalization() (x)
    x = layers.LSTM(256, return_sequences=True, activation="relu", recurrent_dropout=0.1, dropout=0.1) (x)
    x = layers.LayerNormalization() (x)
    #x = layers.Dense(64, activation="relu")(x)
    x = layers.LSTM(128, activation="relu", recurrent_dropout=0.1, dropout=0.1) (x)
    x = layers.LayerNormalization() (x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    
    print(len(data['f'].unique()))
    
    x_pred_layer = layers.Dense(1, name='X') (x)
    y_pred_layer = layers.Dense(1, name ='Y') (x)
    f_pred_layer = layers.Dense(len(data['f'].unique()), name='Floor', activation='softmax') (x)
    
    model = keras.Model(
                    inputs=inputs,
                    outputs=[x_pred_layer, y_pred_layer,
                            f_pred_layer
                            ]
                    )
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss={
            "X": keras.losses.MeanSquaredError(),
            "Y": keras.losses.MeanSquaredError(),
            "Floor": keras.losses.CategoricalCrossentropy(from_logits=True)
            },
            metrics={'X':'mse',
                     'Y':'mse',
                     'Floor':'accuracy'}
            #loss_weights=[1.0, 0.2],
                )
    model.fit(
        {"Input": x_trainX.to_numpy()},
        {"X": y_trainx.to_numpy(), "Y": y_trainy.to_numpy(),
         "Floor":keras.utils.to_categorical(y_trainf, num_classes=None)
        },
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)],
        epochs=2000,
        validation_split=0.2,
        #batch_size=256,
            )
    
#     test_eval = model.evaluate(x=x_testX.to_numpy(), y={"X": y_testx.to_numpy(),
#                                                         "Y": y_testy.to_numpy(),
#                                      "Floor":keras.utils.to_categorical(y_testf.to_numpy(), num_classes=11)
#                                             },verbose=1)
    
    
    test_preds = model.predict(test_data.iloc[:,:-1].to_numpy(), verbose=1)
    
    
    
#     test_predsf = xgb_cF.predict(xgb.DMatrix(test_data.iloc[:,:-1])) #Predicting floor first
    
#     test_data_with_F = test_data.iloc[:,:-1]
#     test_data_with_F['f'] = test_predsf
    
#     test_predsx = xgb_rX.predict(xgb.DMatrix(test_data_with_F))
    
#     test_data_with_F.drop(columns=['f'], inplace=True)
#     test_data_with_F['x'] = test_predsx
#     test_data_with_F['f'] = test_predsf-2
    
#     test_predsy = xgb_rY.predict(xgb.DMatrix(test_data_with_F))
    
    test_preds_f = [np.argmax (x, axis=-1) for x in test_preds[2]]
    test_preds_f = le.inverse_transform(test_preds_f)
    test_preds = pd.DataFrame(np.stack(( test_preds_f, test_preds[0].reshape(-1, ), test_preds[1].reshape(-1, )))).T
    test_preds.columns = ssubm.columns[1:]
    test_preds.index = test_data["site_path_timestamp"]
    
    test_preds["floor"] = test_preds["floor"].astype(int) #Returning floor classes back to original
    
    #Post-processing should go here. Idea is to snap to waypoint if the prediction is to close.
    #*************************
    #Generating dataframe with unique waypoints
    
    #data.drop_duplicates(subset=['x', 'y', 'f'], inplace=True)
    
    #*********End of post processing**********
    
    
    
    
    predictions.append(test_preds)

C:\Users\user\Documents\LightHouseLabs\W09\5a0546857ecc773753327266_1000_train.csv
9296 3403
5
Epoch 1/2000
233/233 [==============================] - 16s 43ms/step - loss: 10792.0089 - X_loss: 5708.0488 - Y_loss: 5080.7808 - Floor_loss: 3.1780 - X_mse: 5708.0488 - Y_mse: 5080.7808 - Floor_accuracy: 0.2010 - val_loss: 89.1646 - val_X_loss: 46.4517 - val_Y_loss: 41.3059 - val_Floor_loss: 1.4071 - val_X_mse: 46.4517 - val_Y_mse: 41.3059 - val_Floor_accuracy: 0.4597
Epoch 2/2000
233/233 [==============================] - 9s 40ms/step - loss: 76.9090 - X_loss: 40.4095 - Y_loss: 35.1442 - Floor_loss: 1.3553 - X_mse: 40.4095 - Y_mse: 35.1442 - Floor_accuracy: 0.5020 - val_loss: 31.7909 - val_X_loss: 16.4218 - val_Y_loss: 14.1628 - val_Floor_loss: 1.2063 - val_X_mse: 16.4218 - val_Y_mse: 14.1628 - val_Floor_accuracy: 0.5758
Epoch 3/2000
233/233 [==============================] - 10s 41ms/step - loss: 31.4556 - X_loss: 16.2738 - Y_loss: 13.9958 - Floor_loss: 1.1860 - X_mse: 16.2738 - Y_mse: 13

In [ ]:
# generate prediction file 
all_preds = pd.concat(predictions)


In [ ]:
all_preds.reset_index(inplace=True)

In [ ]:
all_preds = all_preds.reindex(ssubm.index)


In [ ]:
all_preds.to_csv('submission.csv', index=False)

In [ ]:
all_preds